ref: https://github.com/SuphanutN/Thai-NER-BiLSTMCRF-WordCharEmbedding/blob/master/Thai_NER_WordCharacterEmbedding_Train.ipynb

mycolab: https://colab.research.google.com/drive/1KF_w6wSNSfCQqPEzrJK7NDVJ2uWiKMA2?usp=sharing

# Install 

In [ ]:
!pip install keras==2.1.6

In [ ]:
!pip install dill

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
!pip install XlsxWriter

# Import Library

In [1]:
import sys
print(sys.version)

3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]


In [2]:
# Save / Load File
import dill
import pickle

# Plot Graph
import matplotlib.pyplot as plt

# Sklearn Report
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

# Load Vectors
from gensim.models import KeyedVectors

# Utility
import numpy as np
import time

# Model Utility
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd

# keras
# Navigate to miniconda3/lib/python3.10/site-packages/keras/callbacks.py:18
# Add .abc after collection line: 18 - from collections.abc import Iterable
# https://www.nbshare.io/notebook/589110167/ImportError-cannot-import-name-Iterabler-from-collections-python-3-10/
import keras
# Keras Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.utils import to_categorical
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

# other 
import tensorflow as tf
# tf.disable_v2_behavior()
print(tf.__version__)

Using TensorFlow backend.
/home/crimex/miniconda3/lib/python3.10/site-packages/keras/backend/tensorflow_backend.py:2871: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if training is 1 or training is True:
/home/crimex/miniconda3/lib/python3.10/site-packages/keras/backend/tensorflow_backend.py:2877: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif training is 0 or training is False:
2023-05-19 21:34:33.605511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 21:34:34.086276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19

2.11.0


In [3]:
import tensorflow as tf
print(tf.__version__)


2.11.0


## Path

In [4]:
DATA_PATH='/home/crimex/CRIMEX/BiLSTM-CRF/'
# RAW_PATH = DATA_PATH +'data/'
# MODEL_PATH = DATA_PATH +'model/Keras/WordCharModel/'
W_MODEL_PATH = DATA_PATH +'model/thai2fit/'
Dict_MODEL_PATH = DATA_PATH +'model/dictionary/'

# dataset_name = 'bilstm_train3.data'

In [5]:
typ = 'sent'
path = 'final'
di = '/home/crimex/CRIMEX/Dataset/crf/'+ typ+'/'
train_path = di + 'train_'+ path
test_path = di + 'test_' + path 
save_result_xls = '/home/crimex/CRIMEX/predicted_bert/' + path+ '_'+ typ+'_bilstm.xlsx'
MODEL_PATH = '/home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/'+typ +'/' +path +'/'
save_txt = '/home/crimex/CRIMEX/BiLSTM-CRF/report_' + typ+ '_'+path +'.txt'

# print("directory path       : ", di)
print("MODEL_PATH           : ", MODEL_PATH)
print("\nTrain file           : ", train_path)
print("Test file            : ", test_path)
print("\ntxt eval result      : ", save_txt)
print("xls predicted result : ", save_result_xls)

MODEL_PATH           :  /home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/sent/final/

Train file           :  /home/crimex/CRIMEX/Dataset/crf/sent/train_final
Test file            :  /home/crimex/CRIMEX/Dataset/crf/sent/test_final

txt eval result      :  /home/crimex/CRIMEX/BiLSTM-CRF/report_sent_final.txt
xls predicted result :  /home/crimex/CRIMEX/predicted_bert/final_sent_bilstm.xlsx


# Load raw dataset (NER)

In [6]:
with open(train_path+'.txt', "r") as f:
    train = [line.strip() for line in f.readlines()]
    f.close()
    
with open(test_path+'.txt', "r") as f:
    test = [line.strip() for line in f.readlines()]
    f.close()

In [7]:
import ast

first = True
datatofile = []
train_sents = []
test_sents = []

for tra in train:
    sent = []
    for itm in ast.literal_eval(tra):
        new_itm = itm[:1] + itm[2:]
        sent.append(new_itm)
    datatofile.append(list(sent))
    train_sents.append(list(sent))

for tes in test:
    sent = []
    for itm in ast.literal_eval(tes):
        new_itm = itm[:1] + itm[2:]
        sent.append(new_itm)
    datatofile.append(list(sent))
    test_sents.append(list(sent))

In [8]:
print(len(train_sents))
print(len(test_sents))
# print(train_sents[1])

55410
9336


## Load Thai2Fit Word Embedding

In [9]:
# download file from https://github.com/SuphanutN/Thai-NER-BiLSTMCRF-WordCharEmbedding/blob/master/model/thai2fit/thai2vecNoSym.bin
thai2fit_model = KeyedVectors.load_word2vec_format(W_MODEL_PATH+'thai2vecNoSym.bin',binary=True)
thai2fit_weight = thai2fit_model.vectors
print(W_MODEL_PATH+'thai2vecNoSym.bin')

/home/crimex/CRIMEX/BiLSTM-CRF/model/thai2fit/thai2vecNoSym.bin


## Preprocess Word

In [10]:
word_list=[]
ner_list=[]
thai2dict = {}

for sent in train_sents:
    for word in sent:
        word_list.append(word[0])
        ner_list.append(word[1])
        
for word in thai2fit_model.index_to_key:
    thai2dict[word] = thai2fit_model[word]

word_list.append("pad")
word_list.append("unknown") #Special Token for Unknown words ("UNK")
ner_list.append("pad")

all_words = sorted(set(word_list))
all_ner = sorted(set(ner_list))
all_thai2dict = sorted(set(thai2dict))

word_to_ix = dict((c, i) for i, c in enumerate(all_words)) #convert word to index 
ner_to_ix = dict((c, i) for i, c in enumerate(all_ner)) #convert ner to index
thai2dict_to_ix = dict((c, i) for i, c in enumerate(thai2dict)) #convert thai2fit to index 

ix_to_word = dict((v,k) for k,v in word_to_ix.items()) #convert index to word
ix_to_ner = dict((v,k) for k,v in ner_to_ix.items())  #convert index to ner
ix_to_thai2dict = dict((v,k) for k,v in thai2dict_to_ix.items())  #convert index to thai2fit

n_word = len(word_to_ix)
n_tag = len(ner_to_ix)
n_thai2dict = len(thai2dict_to_ix)
print(n_word)
print(n_tag)
print(n_thai2dict)
print(ner_to_ix)

20487
22
55677
{'B-Action': 0, 'B-Criminal': 1, 'B-Datetime': 2, 'B-Enforcement': 3, 'B-Item': 4, 'B-Location': 5, 'B-Rootcause': 6, 'B-Trigger': 7, 'B-Victim': 8, 'B-worth': 9, 'I-Action': 10, 'I-Criminal': 11, 'I-Datetime': 12, 'I-Enforcement': 13, 'I-Item': 14, 'I-Location': 15, 'I-Rootcause': 16, 'I-Trigger': 17, 'I-Victim': 18, 'I-worth': 19, 'O': 20, 'pad': 21}


## Preprocess Character

In [11]:
chars = set([w_i for w in thai2dict for w_i in w])
char2idx = {c: i + 5 for i, c in enumerate(chars)}

char2idx["pad"] = 0
char2idx["unknown"] = 1
char2idx[" "] = 2

char2idx["$"] = 3
char2idx["#"] = 4
char2idx["!"] = 5
char2idx["%"] = 6
char2idx["&"] = 7
char2idx["*"] = 8
char2idx["+"] = 9
char2idx[","] = 10
char2idx["-"] = 11
char2idx["."] = 12
char2idx["/"] = 13
char2idx[":"] = 14
char2idx[";"] = 15
char2idx["?"] = 16
char2idx["@"] = 17
char2idx["^"] = 18
char2idx["_"] = 19
char2idx["`"] = 20
char2idx["="] = 21
char2idx["|"] = 22
char2idx["~"] = 23
char2idx["'"] = 24
char2idx['"'] = 25

char2idx["("] = 26
char2idx[")"] = 27
char2idx["{"] = 28
char2idx["}"] = 29
char2idx["<"] = 30
char2idx[">"] = 31
char2idx["["] = 32
char2idx["]"] = 33

n_chars = len(char2idx)
print(n_chars)

399


## Save Dictionary for Character and NER

In [12]:
print(Dict_MODEL_PATH+'chardict.pickle')
with open(Dict_MODEL_PATH+'chardict.pickle', 'wb') as chardict: #wb
    pickle.dump(char2idx, chardict)

/home/crimex/CRIMEX/BiLSTM-CRF/model/dictionary/chardict.pickle


In [13]:
print(Dict_MODEL_PATH+'nerdict.pickle')
with open(Dict_MODEL_PATH+'nerdict.pickle', 'wb') as nerdict: #wb
    pickle.dump(ner_to_ix, nerdict)

/home/crimex/CRIMEX/BiLSTM-CRF/model/dictionary/nerdict.pickle


# Set Parameter and Hyper Parameter

In [14]:
max_len = 250
max_len_char = 30

character_LSTM_unit = 32
char_embedding_dim = 32
main_lstm_unit = 256 ## Bidirectional 256 + 256 = 512
lstm_recurrent_dropout = 0.5

train_batch_size = 32
train_epochs = 20

## Mapping Function

In [15]:
def prepare_sequence_word(input_text):
    idxs = list()
    for word in input_text:
        if word in thai2dict:
            idxs.append(thai2dict_to_ix[word])
        else:
            idxs.append(thai2dict_to_ix["unknown"]) #Use UNK tag for unknown word
    return idxs

def prepare_sequence_target(input_label):
    idxs = [ner_to_ix[w] for w in input_label]
    return idxs

### Split word and label

In [16]:
input_sent =[ [ word[0] for word in sent]for sent in train_sents ] #words only
train_targets =[ [ word[1] for word in sent]for sent in train_sents ] #NER only

input_test_sent =[ [ word[0] for word in sent]for sent in test_sents ] #words only
test_targets =[ [ word[1] for word in sent]for sent in test_sents ] #NER only

In [17]:
# input_test_sent[1]

# Prepare Dataset

## Training Dataset

In [18]:
## Word Training
X_word_tr = [prepare_sequence_word(s) for s in input_sent]
X_word_tr = pad_sequences(maxlen=max_len, sequences=X_word_tr, value=thai2dict_to_ix["pad"], padding='post', truncating='post')

## Character Training
X_char_tr = []
for sentence in train_sents:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                if(sentence[i][0][j] in char2idx):
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                else:
                    word_seq.append(char2idx.get("unknown"))
            except:
                word_seq.append(char2idx.get("pad"))
        sent_seq.append(word_seq)
    X_char_tr.append(np.array(sent_seq))

## Sequence Label Training
y_tr = [prepare_sequence_target(s) for s in train_targets]
y_tr = pad_sequences(maxlen=max_len, sequences=y_tr, value=ner_to_ix["pad"], padding='post', truncating='post')
y_tr = [to_categorical(i, num_classes=n_tag) for i in y_tr]

## Testing Dataset

In [19]:
## Word Testing
X_word_te = [prepare_sequence_word(s) for s in input_test_sent]
X_word_te = pad_sequences(maxlen=max_len, sequences=X_word_te, value=thai2dict_to_ix["pad"], padding='post', truncating='post')

## Character Testing
X_char_te = []
for sentence in test_sents:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                if(sentence[i][0][j] in char2idx):
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                else:
                    word_seq.append(char2idx.get("unknown"))
            except:
                word_seq.append(char2idx.get("pad"))    
        sent_seq.append(word_seq)
    X_char_te.append(np.array(sent_seq))

## Sequence Label Testing
y_te = [prepare_sequence_target(s) for s in test_targets]
y_te = pad_sequences(maxlen=max_len, sequences=y_te, value=ner_to_ix["pad"], padding='post', truncating='post')
y_te = [to_categorical(i, num_classes=n_tag) for i in y_te]

# Initial Keras Model

In [20]:
tf.compat.v1.disable_eager_execution()

# Word Input
# -Navigating to, "~/miniconda3/lib/python3.10/site-packages/keras/backend/tensorflow_backend.py".
# Then, replace tensorflow import with i

# import tensorflow.compat.v1 as tf
# tf.compat.v1.disable_eager_execution()

# Now, on line 425 of tensorflow_backend.py: change the hyperparameter dtype from None to tf.float32

# ref: https://github.com/matterport/Mask_RCNN/issues/2075 (Nishanth2708 commented on Aug 26, 2021)
word_in = Input(shape=(max_len,), name='word_input_')

# Word Embedding Using Thai2Fit
word_embeddings = Embedding(input_dim=n_thai2dict,
                            output_dim=400,
                            weights = [thai2fit_weight],input_length=max_len,
                            mask_zero=False,
                            name='word_embedding', trainable=False)(word_in)

# Character Input
char_in = Input(shape=(max_len, max_len_char,), name='char_input')

# Character Embedding
emb_char = TimeDistributed(Embedding(input_dim=n_chars, output_dim=char_embedding_dim, 
                           input_length=max_len_char, mask_zero=False))(char_in)

# Character Sequence to Vector via BiLSTM
char_enc = TimeDistributed(Bidirectional(LSTM(units=character_LSTM_unit, return_sequences=False, recurrent_dropout=lstm_recurrent_dropout)))(emb_char)


# Concatenate All Embedding
all_word_embeddings = concatenate([word_embeddings, char_enc])
all_word_embeddings = SpatialDropout1D(0.3)(all_word_embeddings)

# Main Model BiLSTM
main_lstm = Bidirectional(LSTM(units=main_lstm_unit, return_sequences=True,
                               recurrent_dropout=lstm_recurrent_dropout))(all_word_embeddings)
main_lstm = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# CRF
crf = CRF(n_tag)  # CRF layer
out = crf(main_lstm)  # output

# Model
model = Model([word_in, char_in], out)

# -Navigating to, "~/miniconda3/lib/python3.10/site-packages/keras/optimizers.py".

# add compat.v1. after tf. (line: 757)

# ref: https://itecnote.com/tecnote/python-error-importing-bert-module-tensorflow-_api-v2-train-has-no-attribute-optimizer/
model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

2023-05-19 21:37:12.752006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2023-05-19 21:37:13.039333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 21:37:13.039611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 21:37:13.066599: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-05-19 21:37:13.066621: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to do

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 250, 30)      0                                            
__________________________________________________________________________________________________
word_input_ (InputLayer)        (None, 250)          0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 250, 30, 32)  12768       char_input[0][0]                 
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 250, 400)     22270800    word_input_[0][0]                
__________________________________________________________________________________________________
time_distr

/home/crimex/miniconda3/lib/python3.10/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/crimex/miniconda3/lib/python3.10/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


# Training Model

In [21]:
print(MODEL_PATH)

/home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/sent/final/


In [22]:
filepath=MODEL_PATH+"weights-improvement-{epoch:02d}-{val_crf_viterbi_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_crf_viterbi_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))
                     ], 
                     np.array(y_tr),
                     batch_size=train_batch_size, epochs=train_epochs, verbose=1,callbacks=callbacks_list,
                     validation_data=(
                     [X_word_te,
                     np.array(X_char_te).reshape((len(X_char_te), max_len, max_len_char))
                     ],
                     np.array(y_te))
                   )


Train on 55410 samples, validate on 9336 samples
Epoch 1/20


2023-05-19 21:37:16.952667: W tensorflow/c/c_api.cc:291] Operation '{name:'training/Adam/Variable_59/Assign' id:3882 op device:{requested: '', assigned: ''} def:{{{node training/Adam/Variable_59/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/Variable_59, training/Adam/zeros_59)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


55410/55410 [==============================] - 1537s 28ms/step - loss: 0.0394 - crf_viterbi_accuracy: 0.9869 - val_loss: 0.0131 - val_crf_viterbi_accuracy: 0.9892

Epoch 00001: val_crf_viterbi_accuracy improved from -inf to 0.98918, saving model to /home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/sent/final/weights-improvement-01-0.989.hdf5
Epoch 2/20
55410/55410 [==============================] - 1537s 28ms/step - loss: 0.0038 - crf_viterbi_accuracy: 0.9904 - val_loss: -0.0016 - val_crf_viterbi_accuracy: 0.9904

Epoch 00002: val_crf_viterbi_accuracy improved from 0.98918 to 0.99044, saving model to /home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/sent/final/weights-improvement-02-0.990.hdf5
Epoch 3/20
55410/55410 [==============================] - 1533s 28ms/step - loss: -0.0075 - crf_viterbi_accuracy: 0.9911 - val_loss: -0.0109 - val_crf_viterbi_accuracy: 0.9902

Epoch 00003: val_crf_viterbi_accuracy did not improve from 0.99044
Epoch 4/20
55410/55410 [===============

# Plot Graph

### Accuracy Graph

In [23]:
# hist = pd.DataFrame(history.history)

# plt.style.use("ggplot")
# plt.figure(figsize=(12,12))
# plt.plot(hist["crf_viterbi_accuracy"])
# plt.plot(hist["val_crf_viterbi_accuracy"])
# plt.show()

###  Loss Graph

In [ ]:
# hist = pd.DataFrame(history.history)

# plt.style.use("ggplot")
# plt.figure(figsize=(12,12))
# plt.plot(hist["loss"])
# plt.plot(hist["val_loss"])
# plt.show()

# Save Last Weight Model

In [24]:
save_filepath=MODEL_PATH+"last_weight-50.hdf5"
model.save_weights(save_filepath)

# Load Weight Model

In [25]:
print(MODEL_PATH)

/home/crimex/CRIMEX/BiLSTM-CRF/model/Keras/WordCharModel/sent/final/


In [26]:
# 'str' object has no attribute 'decode'.
# navigate to - ~/miniconda3/lib/python3.10/site-packages/keras/engine/topology.py

# remove .decode('utf-8') line: 2940, 3339, 3343

# ref: https://stackoverflow.com/questions/28583565/str-object-has-no-attribute-decode-python-3-error

# load_filepath = MODEL_PATH + path + "/weights-improvement-18-0.991.hdf5" 
load_filepath=MODEL_PATH+'weights-improvement-06-0.991.hdf5'
model.load_weights(load_filepath)

# Prediction

In [27]:
pred_model = model.predict(
    [X_word_te,
     np.array(X_char_te).reshape(
                                 (len(X_char_te),
                                  max_len, 
                                  max_len_char))
    ], 
    verbose=1)

2023-05-20 06:35:54.992202: W tensorflow/c/c_api.cc:291] Operation '{name:'crf_1/cond/Identity' id:1165 op device:{requested: '', assigned: ''} def:{{{node crf_1/cond/Identity}} = Identity[T=DT_FLOAT, _has_manual_control_dependencies=true](crf_1/cond)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


9336/9336 [==============================] - 59s 6ms/step


In [28]:
y_pred = []
y_test = []

for i in range(0,len(pred_model)):
    try:
        out = np.argmax(pred_model[i], axis=-1)
        true = np.argmax(y_te[i], axis=-1)
        revert_pred=[ix_to_ner[i] for i in out]
        revert_true=[ix_to_ner[i] for i in true]
        y_pred.append(revert_pred)
        y_test.append(revert_true)
    except:
        print (i)

In [29]:
import sklearn.metrics
import numpy
from itertools import chain

flatten_true_predictions = []
flatten_true_labels = []

for i in range (0,len(y_test)):
#     print(y_test[i])
    for k in range(0,len(y_test[i])):
#         print(y_test[i][k])
        if y_test[i][k] != 'pad'and y_pred[i][k] != 'pad': 
            flatten_true_predictions.append(y_pred[i][k])
            flatten_true_labels.append(y_test[i][k])

In [30]:
from sklearn.metrics import matthews_corrcoef, confusion_matrix
tag_map = {'B-Criminal': 'Criminal',
           'I-Criminal': 'Criminal',
           'B-Action': 'Action',
           'I-Action': 'Action',
           'B-Location': 'Location',
           'I-Location': 'Location',
           'B-Item': 'Item',
           'I-Item': 'Item',
           'B-Victim': 'Victim',
           'I-Victim': 'Victim',
           'B-worth': 'Worth',
           'I-worth': 'Worth',
           'B-Datetime': 'Datetime',
           'I-Datetime': 'Datetime',
           'B-Enforcement': 'Enforcement',
           'I-Enforcement': 'Enforcement',
           'B-Rootcause': 'Rootcause',
           'I-Rootcause': 'Rootcause',
           'B-Trigger': 'Trigger',
           'I-Trigger': 'Trigger',
           'O': 'O'}

# Convert the original tags to merged tags
merged_true_labels = [tag_map[tag] for tag in flatten_true_labels]
merged_true_predictions = [tag_map[tag] for tag in flatten_true_predictions]

ny_true = np.array(merged_true_labels)
ny_pred = np.array(merged_true_predictions)

# mcc report (str)
mcc_report = "+--------------+-----------------------+\n"
mcc_report += f"| MCC report \n"
mcc_report += "+--------------+-----------------------+\n"

# Calculate MCC for all labels
mcc_all = matthews_corrcoef(ny_true, ny_pred)
# print("MCC (all labels):", mcc_all)

# Calculate MCC for each label
for label in np.unique(ny_true):
    y_true_label = np.where(ny_true == label, 1, 0)
    y_pred_label = np.where(ny_pred == label, 1, 0)
    mcc_label = matthews_corrcoef(y_true_label, y_pred_label)
#     print(f"MCC (label {label}):", mcc_label)
    mcc_report += f"| {label:<12} |  {mcc_label}\n"
    
mcc_report += "+---------------+-----------------------+\n"
mcc_report += f"| {'Overall':<12} |  {mcc_all}\n"
mcc_report += "+---------------+-----------------------+\n"
print(mcc_report)

+--------------+-----------------------+
| MCC report 
+--------------+-----------------------+
| Action       |  0.07113231946796235
| Criminal     |  0.6303508260494857
| Datetime     |  0.3389507125930282
| Enforcement  |  0.5380707561567631
| Item         |  0.6405760232862372
| Location     |  0.7037548309320654
| O            |  0.544557129002601
| Rootcause    |  0.08885301785857623
| Trigger      |  0.038553571697930174
| Victim       |  0.5573909124204773
| Worth        |  0.4364316529377991
+---------------+-----------------------+
| Overall      |  0.5284769960356585
+---------------+-----------------------+



In [31]:
from sklearn.metrics import accuracy_score
accuracy_all = dict()
for label in np.unique(ny_true):
    y_true_label = np.where(ny_true == label, 1, 0)
    y_pred_label = np.where(ny_pred == label, 1, 0)
    accuracy = accuracy_score(y_true_label, y_pred_label)
    accuracy_all[label] = accuracy

# print("Accuracy (all labels):", np.mean(list(accuracy_all.values())))

# acc report (str)
acc_report = "+--------------+-----------------------+\n"
acc_report += f"| ACC report \n"
acc_report += "+--------------+-----------------------+\n"

for label, accuracy in accuracy_all.items():
#     print(f"Accuracy (label {label}):", accuracy)
    acc_report += f"| {label:<12} |  {accuracy}\n"
    
acc_report += "+---------------+-----------------------+\n"
acc_report += f"| {'Overall':<12} |  {np.mean(list(accuracy_all.values()))}\n"
acc_report += "+---------------+-----------------------+\n"
print(acc_report)

+--------------+-----------------------+
| ACC report 
+--------------+-----------------------+
| Action       |  0.9868824384781805
| Criminal     |  0.9856917753041258
| Datetime     |  0.9912194924228223
| Enforcement  |  0.9829304500706798
| Item         |  0.9857779083848021
| Location     |  0.9912042924674087
| O            |  0.9023301531648841
| Rootcause    |  0.9936261520299541
| Trigger      |  0.9924456221595084
| Victim       |  0.9887824329048635
| Worth        |  0.9859248412871322
+---------------+-----------------------+
| Overall      |  0.9806195962431236
+---------------+-----------------------+



In [32]:
from sklearn.metrics import classification_report
report = classification_report(ny_true, ny_pred, labels=np.unique(ny_true))
print(report)

              precision    recall  f1-score   support

      Action       0.18      0.03      0.05      2335
    Criminal       0.66      0.62      0.64      4002
    Datetime       0.52      0.23      0.32      1758
 Enforcement       0.49      0.61      0.54      3277
        Item       0.66      0.64      0.65      4043
    Location       0.82      0.61      0.70      3321
           O       0.92      0.97      0.94    170062
   Rootcause       0.33      0.02      0.05      1228
     Trigger       0.45      0.00      0.01      1490
      Victim       0.61      0.52      0.56      2723
       Worth       0.61      0.32      0.42      3130

    accuracy                           0.89    197369
   macro avg       0.57      0.42      0.44    197369
weighted avg       0.88      0.89      0.88    197369



In [33]:
print(save_txt)
with open(save_txt, 'w') as file:
    
    defi = 'Train by ' + str(len(y_test)) + ' sents &&& Test by ' + str(len(y_pred)) + ' sents\n'
    file.write(defi)

#     file.write('classification_report\n\n')
#     file.write(classi_report)
    file.write('\n-----------------------------------------------------------------\n\n')
    file.write('classification_report\n')
    file.write(report)
    file.write('\n-----------------------------------------------------------------\n\n\n')
    file.write(mcc_report)
    file.write('\n-----------------------------------------------------------------\n\n\n')
    file.write(acc_report)
    
    file.close()

/home/crimex/CRIMEX/BiLSTM-CRF/report_sent_final.txt


In [35]:
import xlsxwriter
print(save_result_xls)
workbook = xlsxwriter.Workbook(save_result_xls)
worksheet = workbook.add_worksheet("My sheet")
row = 0
col = 0
for i in y_pred:
    str1 = str(i)
    worksheet.write(row, 0 , str1)
    row += 1
row = 0
for i in y_test:
    str2 = str(i)
    worksheet.write(row, 1 , str2)
    row += 1
workbook.close()

/home/crimex/CRIMEX/predicted_bert/final_sent_bilstm.xlsx
